In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf


In [3]:
df = pd.read_csv('SOLUSTDAtas_tratado.csv')
df.head()

,date,open,high,low,close
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.4+ MB
None


# Preprocesado de Datos

In [5]:
tamanio = df.shape[0]

In [6]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_train

,date,open,high,low,close
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25
...,...,...,...,...,...
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95
25479,2023-07-12 00:00:00,21.95,22.10,21.90,22.05


In [7]:
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_vali

,date,open,high,low,close
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95
25485,2023-07-12 06:00:00,21.95,22.05,21.90,22.00
...,...,...,...,...,...
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30
32759,2024-05-10 08:00:00,153.30,155.10,153.15,154.95


In [8]:
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]
df_test

,date,open,high,low,close
32761,2024-05-10 10:00:00,154.35,154.5,153.45,154.10
32762,2024-05-10 11:00:00,154.10,154.8,153.25,154.15
32763,2024-05-10 12:00:00,154.15,154.3,153.25,154.15
32764,2024-05-10 13:00:00,154.15,155.2,153.00,155.05
32765,2024-05-10 14:00:00,155.05,155.4,153.10,153.30
...,...,...,...,...,...
36395,2024-10-08 20:00:00,143.35,143.9,142.35,142.95
36396,2024-10-08 21:00:00,142.95,144.1,142.25,143.75
36397,2024-10-08 22:00:00,143.75,144.5,143.35,144.50
36398,2024-10-08 23:00:00,144.50,144.7,144.05,144.25


In [9]:
df_valitest = pd.concat([df_vali, df_test], axis=0)

Numero de horas que se utilizan en la predicción

In [10]:
numhorasconst = 4

# Redes neuronales LSTM

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import LSTM

In [12]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps, 3])  
    return np.array(X), np.array(y)

In [13]:
def preparar_datosLSTM(df, numhoras):
    return create_sequences(df[['open', 'high', 'low', 'close']].values, numhoras)

In [14]:
def evalRedLSTM(ytest, y_pred):
    y_pred = y_pred.flatten()
    suma = 0
    n = len(y_pred)
    for i in range(0,n):
        suma = abs(y_pred[i] - ytest[i])/ytest[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [15]:
def opti_redes_LSTM(epoch_array, batch_array, X_trainLSTM, y_trainLSTM, X_valiLSTM, y_valiLSTM, X_testLSTM, y_testLSTM, numhoras):
    epoch_best = 0
    bacth_best = 0
    best_model = None
    best = float('inf')
    results = []
    for e in epoch_array:
        for b in batch_array:
            for i in range(15):
                with tf.device('/CPU:0'):
                    modelLSTM = Sequential()
                    modelLSTM.add(LSTM(128, activation='relu', input_shape=(numhoras, 4)))
                    modelLSTM.add(Dense(1))
                    modelLSTM.compile(optimizer='adam', loss='mape')
                    historyLSTM = modelLSTM.fit(X_trainLSTM, y_trainLSTM, epochs=e, batch_size=b, validation_data=(X_valiLSTM, y_valiLSTM), shuffle=False)
                    y_pred = modelLSTM.predict(X_testLSTM)
                    valor = evalRedLSTM(y_testLSTM, y_pred)
                    print("epoch: "+str(e)+", batch_size: "+str(b)+", value: "+str(valor))
                    results.append([e, b, valor])
                    if valor < best:
                        best = valor
                        epoch_best = e
                        bacth_best = b
                        best_model = modelLSTM
                        if valor < 0.75:
                            cadena_guardado = "ModelosLSTMOptiMoreDataIMC/mi_modelo_LSTMOpti_e"+str(e)+"_b"+str(b)+"_v"+str(round(valor, 3))
                            best_model.save(cadena_guardado+".keras")
    df_results = pd.DataFrame(results, columns=["epoch", "batch_size", "value"])
    return epoch_best, bacth_best, best, best_model, df_results

In [16]:
#BATCH_ARRAY = [4, 8, 12, 16, 32, 64, 128, 256]
BATCH_ARRAY = [128, 256]

In [17]:
def opti_rLSTM_h(h_array, epoch_array, batch_array):
    best = 100
    epoch_best = 0
    bacth_best = 0
    h_best = 0
    best_model = None
    for i in h_array:
        Xtrain, ytrain = preparar_datosLSTM(df_train, i)
        Xvali, yvali = preparar_datosLSTM(df_vali, i)
        Xtest, ytest = preparar_datosLSTM(df_test, i)
        valores = opti_redes_LSTM(epoch_array, batch_array, Xtrain, ytrain, Xvali, yvali, Xtest, ytest, i)
        df = valores[4]
        csv_filename = "lstmH" + str(i) + ".csv"
        df.to_csv(csv_filename, index=False)
    return best, epoch_best, bacth_best, h_best, best_model

In [ ]:
#data = opti_rLSTM_h([1, 3, 5, 7, 10, 12, 14, 18, 21], [4, 6, 10, 14, 20, 40], BATCH_ARRAY)
data = opti_rLSTM_h([1, 3], [4, 6], BATCH_ARRAY)